In [4]:
import pandas as pd
import json

findings = pd.read_json("data/findings/combined.jsonl", lines=True)
print(f"Number of lines in combined.jsonl: {len(findings)}")
print(f"Total number of individual findings: {sum(len(row['findings']) for _, row in findings.iterrows())}")

Number of lines in combined.jsonl: 10938
Total number of individual findings: 98522


In [8]:
research = pd.read_json("data/research_used.jsonl", lines=True)
research.set_index("doi", inplace=True)
research["pubdate"] = research["pubdate"].apply(lambda x: int(x.replace("-", "")))
print(len(research))

10027


In [13]:
from tqdm import tqdm
all_records = pd.DataFrame(columns=['text', 'doi', 'pubdate']).astype({
    'text': 'string',
    'doi': 'string',
    'pubdate': 'int'
})

for row in tqdm(findings.itertuples(), total=len(findings)):
    doi = row.doi
    research_record = research.loc[doi]
    pubdate = research_record["pubdate"]
    texts = row.findings
    new_rows = [
        {"text": text, "doi": doi, "pubdate": pubdate} for text in texts
    ]
    all_records = pd.concat([all_records, pd.DataFrame(new_rows)], ignore_index=True)

print(f"Assembled {len(all_records)} records.")


100%|██████████| 10938/10938 [00:06<00:00, 1623.16it/s]

Assembled 98522 records.


In [15]:
# Write out all records to a JSONL file
all_records.to_json("data/research_contributions.jsonl", orient="records", lines=True)